In [1]:
with open('./inputs/day11.txt', 'r') as f:
    stones = list(map(int, f.read().split()))

orig_stones = stones.copy()

stone_map = {}
def blink_stone(stone):
    if stone_map.get(stone) is None:
        if stone == 0:
            stone_map[stone] = [1]
        elif len(str(stone)) % 2 == 0:
            stone_map[stone] = [int(str(stone)[:len(str(stone)) // 2]), int(str(stone)[len(str(stone)) // 2:])]
        else:
            stone_map[stone] = [2024 * stone]

    return stone_map[stone]

def blink_stones(input_stones):
    output_stones = []
    for stone in input_stones:
        output_stones += blink_stone(stone)

    return output_stones

for i in range(25):
    stones = blink_stones(stones)

print('Answer to Day 11, Part 1:', len(stones))

stone_time_map = {}
def blink_length(stone, num_blinks):
    if num_blinks == 0:
        return 1
    elif stone_time_map.get((stone, num_blinks)) is None:
        ans = sum(blink_length(s, num_blinks-1) for s in blink_stone(stone))
        stone_time_map[(stone, num_blinks)] = ans
        return ans
    else:
        return stone_time_map[(stone, num_blinks)]

print('Answer to Day 11, Part 2:', sum(blink_length(s, num_blinks=75) for s in orig_stones))

Answer to Day 11, Part 1: 203228
Answer to Day 11, Part 2: 240884656550923
